In [ ]:
#default_exp experiments.cult_experiments

In [ ]:
from cult.core.cult import CULT, CULTTrainer
from cult.core.models import FCEncoder, FCDecoder
from cult.core.datasets.moving_mnist import CommonMNIST, CommonFashionMNIST
from cult.core.utils import show_batch, rec_likelihood
from cult.config import DATA_PATH

import torch
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from datetime import datetime as dt

# Cult Experiments

In [ ]:
batch_size = 64

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
mnist_train = CommonMNIST(DATA_PATH, train=True, transform=ToTensor())
mnist_test = CommonMNIST(DATA_PATH, train=False, transform=ToTensor())
mnist_train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
mnist_test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=True)

mnist_test_batch, _ = iter(mnist_test_loader).next()

fashion_train = CommonFashionMNIST(DATA_PATH, train=True, transform=ToTensor())
fashion_test = CommonFashionMNIST(DATA_PATH, train=False, transform=ToTensor())
fashion_train_loader = DataLoader(fashion_train, batch_size=batch_size, shuffle=True)
fashion_test_loader = DataLoader(fashion_test, batch_size=batch_size, shuffle=True)

fashion_test_batch, _ = iter(fashion_test_loader).next()


## FashionMNIST -> MNIST

In [ ]:
optimizer = torch.optim.Adam
lr = 1e-3
kl_scale = 1
e_prox_scale = 1 #might need to scale?
d_prox_scale = 1 #might need to scale
encoder_type = FCEncoder
decoder_type = FCDecoder
final_size = 50
latents = 16
max_envs = 7
atyp_min = .25
atyp_max = 1.4
env_optim = torch.optim.Adam
env_lr=1e-4
env_epochs = 1 #increasing due to poor performance
replay_batch_size = 64
steps_per_save = 2500

In [ ]:
foo = CULTTrainer('foo', optimizer, lr, kl_scale, e_prox_scale, d_prox_scale, encoder_type, decoder_type, final_size, latents, max_envs, atyp_min, atyp_max, env_optim, env_lr, env_epochs, replay_batch_size, steps_per_save, device)

In [ ]:
foo.model.steps

0

In [ ]:
len(mnist_train_loader) * 10

9380

In [ ]:
for exp_num in range(5):
    epochs = 10
    name = f"fash_to_mnist_{exp_num}"
    cult_trainer = CULTTrainer(name, optimizer, lr, kl_scale, e_prox_scale, d_prox_scale, encoder_type, decoder_type, final_size, latents, max_envs, atyp_min, atyp_max, env_optim, env_lr, env_epochs, replay_batch_size, steps_per_save, device)
    cult_trainer.train(fashion_train_loader, epochs, [fashion_test_batch, mnist_test_batch])
    cult_trainer.train(mnist_train_loader, epochs, [fashion_test_batch, mnist_test_batch])
    cult_trainer.train_latent_classifiers([fashion_train_loader, mnist_train_loader], [fashion_test_loader, mnist_test_loader], ["fashion_mnist", "mnist"], [10, 10], epochs=3, lr=1e-2, verbose=False)
    cult_trainer.model.eval()
    cult_trainer.model.env_net.eval()
    fashion_rec_loss = cult_trainer.rec_loss(fashion_test_loader)
    mnist_rec_loss = cult_trainer.rec_loss(mnist_test_loader)
    cult_trainer.writer.add_scalar("eval/fashion_rec_loss", fashion_rec_loss)
    cult_trainer.writer.add_scalar("eval/mnist_recc_loss", mnist_rec_loss)
    fashion_env_acc = cult_trainer.env_accuracy(fashion_test_loader, 0)
    mnist_env_acc = cult_trainer.env_accuracy(mnist_test_loader, 1)
    cult_trainer.writer.add_scalar("eval/fashion_env_acc", fashion_env_acc)
    cult_trainer.writer.add_scalar("eval/mnist_env_acc", mnist_env_acc)

epoch: 0, loss=882.3352661132812, rec_loss=323.55364990234375, total_div_loss=11.868779182434082
epoch: 1, loss=842.9118041992188, rec_loss=280.90960693359375, total_div_loss=15.95385456085205
epoch: 2, loss=833.99072265625, rec_loss=271.1559753417969, total_div_loss=16.97806167602539
epoch: 3, loss=831.0304565429688, rec_loss=267.6246643066406, total_div_loss=17.66572380065918
epoch: 4, loss=829.6016235351562, rec_loss=265.91729736328125, total_div_loss=18.010046005249023
epoch: 5, loss=828.77587890625, rec_loss=264.83648681640625, total_div_loss=18.28383445739746
epoch: 6, loss=827.836669921875, rec_loss=264.0345764160156, total_div_loss=18.204912185668945
epoch: 7, loss=827.3883056640625, rec_loss=263.50250244140625, total_div_loss=18.299375534057617
epoch: 8, loss=827.0670776367188, rec_loss=263.1199951171875, total_div_loss=18.399784088134766
epoch: 9, loss=826.61328125, rec_loss=262.71893310546875, total_div_loss=18.371368408203125
epoch: 0, loss=480.257568359375, rec_loss=180.79

100%|██████████| 7/7 [02:35<00:00, 22.25s/it]


epoch: 0, loss=881.1354370117188, rec_loss=323.1217956542969, total_div_loss=11.409282684326172
epoch: 1, loss=840.3844604492188, rec_loss=280.5863342285156, total_div_loss=14.154522895812988
epoch: 2, loss=832.8284912109375, rec_loss=272.1751403808594, total_div_loss=15.182563781738281
epoch: 3, loss=830.2005004882812, rec_loss=269.0063171386719, total_div_loss=15.82284164428711
epoch: 4, loss=828.2120971679688, rec_loss=266.6905517578125, total_div_loss=16.234737396240234
epoch: 5, loss=827.1129150390625, rec_loss=265.1929931640625, total_div_loss=16.66802215576172
epoch: 6, loss=826.1986083984375, rec_loss=264.15045166015625, total_div_loss=16.832855224609375
epoch: 7, loss=825.40869140625, rec_loss=263.3125305175781, total_div_loss=16.91683006286621
epoch: 8, loss=825.094482421875, rec_loss=262.8172912597656, total_div_loss=17.133073806762695
epoch: 9, loss=824.730224609375, rec_loss=262.4314880371094, total_div_loss=17.17717933654785
epoch: 0, loss=470.5045166015625, rec_loss=180.

100%|██████████| 7/7 [02:39<00:00, 22.72s/it]


epoch: 0, loss=881.8001708984375, rec_loss=322.51666259765625, total_div_loss=12.228106498718262
epoch: 1, loss=841.5650024414062, rec_loss=280.15765380859375, total_div_loss=14.926261901855469
epoch: 2, loss=833.6214599609375, rec_loss=271.47430419921875, total_div_loss=15.76183032989502
epoch: 3, loss=830.1779174804688, rec_loss=267.527099609375, total_div_loss=16.33484649658203
epoch: 4, loss=828.6974487304688, rec_loss=265.6979064941406, total_div_loss=16.721275329589844
epoch: 5, loss=827.6818237304688, rec_loss=264.5208435058594, total_div_loss=16.94794273376465
epoch: 6, loss=827.0484008789062, rec_loss=263.6932067871094, total_div_loss=17.18001937866211
epoch: 7, loss=826.3844604492188, rec_loss=262.96099853515625, total_div_loss=17.270259857177734
epoch: 8, loss=826.1036987304688, rec_loss=262.4926452636719, total_div_loss=17.47995948791504
epoch: 9, loss=825.6807250976562, rec_loss=262.0513916015625, total_div_loss=17.549592971801758
epoch: 0, loss=471.3250732421875, rec_loss

100%|██████████| 7/7 [02:47<00:00, 23.96s/it]


epoch: 0, loss=881.3013305664062, rec_loss=322.6269226074219, total_div_loss=12.005391120910645
epoch: 1, loss=840.9617919921875, rec_loss=279.283203125, total_div_loss=15.9032621383667
epoch: 2, loss=833.8262939453125, rec_loss=271.3272705078125, total_div_loss=16.869237899780273
epoch: 3, loss=830.8634643554688, rec_loss=267.99371337890625, total_div_loss=17.312910079956055
epoch: 4, loss=828.9489135742188, rec_loss=265.96630859375, total_div_loss=17.44205665588379
epoch: 5, loss=827.3142700195312, rec_loss=264.4073791503906, total_div_loss=17.378774642944336
epoch: 6, loss=826.317626953125, rec_loss=263.4772644042969, total_div_loss=17.303197860717773
epoch: 7, loss=825.4873657226562, rec_loss=262.6854553222656, total_div_loss=17.28038787841797
epoch: 8, loss=825.0408935546875, rec_loss=262.201171875, total_div_loss=17.330080032348633
epoch: 9, loss=824.6445922851562, rec_loss=261.77069091796875, total_div_loss=17.38921356201172
epoch: 0, loss=472.3203430175781, rec_loss=180.3636322

100%|██████████| 7/7 [02:40<00:00, 22.92s/it]


## MNIST -> Fashion MNIST

In [ ]:
optimizer = torch.optim.Adam
lr = 1e-3
kl_scale = 1
e_prox_scale = 1 #might need to scale?
d_prox_scale = 1 #might need to scale
encoder_type = FCEncoder
decoder_type = FCDecoder
final_size = 50
latents = 16
max_envs = 7
atyp_min = .25
atyp_max = 1.4
env_optim = torch.optim.Adam
env_lr=1e-4
env_epochs = 1 #increasing due to poor performance
replay_batch_size = 64
steps_per_save = 2500

In [ ]:
for exp_num in range(5):
    epochs = 10
    name = f"mnist_to_fash_{exp_num}"
    cult_trainer = CULTTrainer(name, optimizer, lr, kl_scale, e_prox_scale, d_prox_scale, encoder_type, decoder_type, final_size, latents, max_envs, atyp_min, atyp_max, env_optim, env_lr, env_epochs, replay_batch_size, steps_per_save, device)
    cult_trainer.train(mnist_train_loader, epochs, [mnist_test_batch, fashion_test_batch])
    cult_trainer.train(fashion_train_loader, epochs, [mnist_test_batch, fashion_test_batch])
    cult_trainer.train_latent_classifiers([mnist_train_loader, fashion_train_loader], [mnist_test_loader, fashion_test_loader], ["mnist", "fashion_mnist"], [10, 10], epochs=3, lr=1e-2)
    cult_trainer.model.eval()
    fashion_rec_loss = cult_trainer.rec_loss(fashion_test_loader)
    mnist_rec_loss = cult_trainer.rec_loss(mnist_test_loader)
    cult_trainer.writer.add_scalar("eval/fashion_rec_loss", fashion_rec_loss)
    cult_trainer.writer.add_scalar("eval/mnist_recc_loss", mnist_rec_loss)
    fashion_env_acc = cult_trainer.env_accuracy(fashion_test_loader, 1)
    mnist_env_acc = cult_trainer.env_accuracy(mnist_test_loader, 0)
    cult_trainer.writer.add_scalar("eval/fashion_env_acc", fashion_env_acc)
    cult_trainer.writer.add_scalar("eval/mnist_env_acc", mnist_env_acc)


epoch: 0, loss=764.3453979492188, rec_loss=210.51756286621094, total_div_loss=6.841984272003174
epoch: 1, loss=728.7280883789062, rec_loss=172.51577758789062, total_div_loss=9.332086563110352
epoch: 2, loss=721.8748168945312, rec_loss=163.42025756835938, total_div_loss=11.954620361328125
epoch: 3, loss=717.7255249023438, rec_loss=158.1334686279297, total_div_loss=13.453836441040039
epoch: 4, loss=715.6884765625, rec_loss=154.83721923828125, total_div_loss=14.420963287353516
epoch: 5, loss=713.866943359375, rec_loss=152.75584411621094, total_div_loss=15.089506149291992
epoch: 6, loss=713.3990478515625, rec_loss=151.5978240966797, total_div_loss=15.527010917663574
epoch: 7, loss=712.3328247070312, rec_loss=150.55862426757812, total_div_loss=15.88389778137207
epoch: 8, loss=711.7504272460938, rec_loss=149.72372436523438, total_div_loss=16.069578170776367
epoch: 9, loss=711.3583984375, rec_loss=149.1879425048828, total_div_loss=16.326169967651367
epoch: 0, loss=492.3997497558594, rec_loss=

  0%|          | 0/7 [00:00<?, ?it/s]

epoch 0 acc tensor(0.8874)
epoch 1 acc tensor(0.9027)
epoch 2 acc tensor(0.9029)
epoch 0 acc tensor(0.7150)
epoch 1 acc tensor(0.7381)
epoch 2 acc tensor(0.7493)


 14%|█▍        | 1/7 [00:27<02:45, 27.51s/it]

epoch 0 acc tensor(0.8864)
epoch 1 acc tensor(0.8962)
epoch 2 acc tensor(0.9002)
epoch 0 acc tensor(0.7235)
epoch 1 acc tensor(0.7386)
epoch 2 acc tensor(0.7509)


...

 71%|███████▏  | 5/7 [02:17<00:55, 27.58s/it]

epoch 0 acc tensor(0.8643)
epoch 1 acc tensor(0.8754)
epoch 2 acc tensor(0.8842)
epoch 0 acc tensor(0.7666)
epoch 1 acc tensor(0.7676)
epoch 2 acc tensor(0.7767)


 86%|████████▌ | 6/7 [02:45<00:27, 27.62s/it]

epoch 0 acc tensor(0.8527)
epoch 1 acc tensor(0.8664)
epoch 2 acc tensor(0.8613)
epoch 0 acc tensor(0.7515)
epoch 1 acc tensor(0.7688)
epoch 2 acc tensor(0.7733)


100%|██████████| 7/7 [03:12<00:00, 27.46s/it]


## Static Update: FashionMNIST -> MNIST

In [ ]:
optimizer = torch.optim.Adam
lr = 1e-3
kl_scale = 1
e_prox_scale = 1 #might need to scale?
d_prox_scale = 1 #might need to scale
encoder_type = FCEncoder
decoder_type = FCDecoder
final_size = 50
latents = 16
max_envs = 7
atyp_min = .25
atyp_max = 1.4
env_optim = torch.optim.Adam
env_lr=1e-4
env_epochs = 1 #increasing due to poor performance
replay_batch_size = 64
steps_per_save = 2500
steps_per_reset = 500 # if None, replay model only updated at new environment

In [ ]:
for exp_num in range(5):
    epochs = 10
    name = f"fixed_fash_to_mnist_{exp_num}"
    cult_trainer = CULTTrainer(name, optimizer, lr, kl_scale, e_prox_scale, d_prox_scale, encoder_type, decoder_type, final_size, latents, max_envs, atyp_min, atyp_max, env_optim, env_lr, env_epochs, replay_batch_size, steps_per_save, device, steps_per_reset)
    cult_trainer.train(fashion_train_loader, epochs, [fashion_test_batch, mnist_test_batch])
    cult_trainer.train(mnist_train_loader, epochs, [fashion_test_batch, mnist_test_batch])
    cult_trainer.train_latent_classifiers([fashion_train_loader, mnist_train_loader], [fashion_test_loader, mnist_test_loader], ["fashion_mnist", "mnist"], [10, 10], epochs=3, lr=1e-2)
    cult_trainer.model.eval()
    fashion_rec_loss = cult_trainer.rec_loss(fashion_test_loader)
    mnist_rec_loss = cult_trainer.rec_loss(mnist_test_loader)
    cult_trainer.writer.add_scalar("eval/fashion_rec_loss", fashion_rec_loss)
    cult_trainer.writer.add_scalar("eval/mnist_recc_loss", mnist_rec_loss)
    fashion_env_acc = cult_trainer.env_accuracy(fashion_test_loader, 0)
    mnist_env_acc = cult_trainer.env_accuracy(mnist_test_loader, 1)
    cult_trainer.writer.add_scalar("eval/fashion_env_acc", fashion_env_acc)
    cult_trainer.writer.add_scalar("eval/mnist_env_acc", mnist_env_acc)


epoch: 0, loss=796.3815307617188, rec_loss=321.3736877441406, total_div_loss=13.189351081848145
epoch: 1, loss=614.6005859375, rec_loss=278.6806335449219, total_div_loss=15.87909984588623
epoch: 2, loss=585.7134399414062, rec_loss=270.21435546875, total_div_loss=17.573097229003906
epoch: 3, loss=573.8211059570312, rec_loss=266.4117736816406, total_div_loss=18.862581253051758
epoch: 4, loss=567.7890014648438, rec_loss=264.202880859375, total_div_loss=19.71156883239746
epoch: 5, loss=564.7792358398438, rec_loss=262.7940979003906, total_div_loss=20.17998504638672
epoch: 6, loss=563.1238403320312, rec_loss=261.83990478515625, total_div_loss=20.57794761657715
epoch: 7, loss=561.0931396484375, rec_loss=260.9774169921875, total_div_loss=20.922508239746094
epoch: 8, loss=559.92431640625, rec_loss=260.3429870605469, total_div_loss=21.067846298217773
epoch: 9, loss=558.4791259765625, rec_loss=259.9775695800781, total_div_loss=21.276567459106445
epoch: 0, loss=438.9420166015625, rec_loss=181.1367

  0%|          | 0/7 [00:00<?, ?it/s]

epoch 0 acc tensor(0.7504)
epoch 1 acc tensor(0.7710)
epoch 2 acc tensor(0.7737)
epoch 0 acc tensor(0.7251)
epoch 1 acc tensor(0.7545)
epoch 2 acc tensor(0.7658)


 14%|█▍        | 1/7 [00:34<03:26, 34.38s/it]

epoch 0 acc tensor(0.7510)
epoch 1 acc tensor(0.7576)
epoch 2 acc tensor(0.7742)
epoch 0 acc tensor(0.6426)
epoch 1 acc tensor(0.6915)
epoch 2 acc tensor(0.7246)


...

 71%|███████▏  | 5/7 [02:49<01:09, 34.59s/it]

epoch 0 acc tensor(0.7116)
epoch 1 acc tensor(0.7074)
epoch 2 acc tensor(0.7287)
epoch 0 acc tensor(0.8116)
epoch 1 acc tensor(0.8249)
epoch 2 acc tensor(0.8448)


 86%|████████▌ | 6/7 [03:26<00:35, 35.31s/it]

epoch 0 acc tensor(0.7009)
epoch 1 acc tensor(0.7151)
epoch 2 acc tensor(0.7276)
epoch 0 acc tensor(0.8367)
epoch 1 acc tensor(0.8544)
epoch 2 acc tensor(0.8670)


100%|██████████| 7/7 [04:00<00:00, 34.36s/it]


## No Replay: FashionMNIST -> MNIST

In [ ]:
optimizer = torch.optim.Adam
lr = 1e-3
kl_scale = 1
e_prox_scale = 0 
d_prox_scale = 0 
encoder_type = FCEncoder
decoder_type = FCDecoder
final_size = 50
latents = 16
max_envs = 7
atyp_min = .25
atyp_max = 1.4
env_optim = torch.optim.Adam
env_lr=1e-4
env_epochs = 1
replay_batch_size = 64
steps_per_save = 2500

In [ ]:
for exp_num in range(5):
    epochs = 10
    name = f"no_replay_fash_to_mnist_{exp_num}"
    cult_trainer = CULTTrainer(name, optimizer, lr, kl_scale, e_prox_scale, d_prox_scale, encoder_type, decoder_type, final_size, latents, max_envs, atyp_min, atyp_max, env_optim, env_lr, env_epochs, replay_batch_size, steps_per_save, device)
    cult_trainer.train(fashion_train_loader, epochs, [fashion_test_batch, mnist_test_batch])
    cult_trainer.train(mnist_train_loader, epochs, [fashion_test_batch, mnist_test_batch])
    cult_trainer.train_latent_classifiers([fashion_train_loader, mnist_train_loader], [fashion_test_loader, mnist_test_loader], ["fashion_mnist", "mnist"], [10, 10], epochs=3, lr=1e-2)
    cult_trainer.model.eval()
    fashion_rec_loss = cult_trainer.rec_loss(fashion_test_loader)
    mnist_rec_loss = cult_trainer.rec_loss(mnist_test_loader)
    cult_trainer.writer.add_scalar("eval/fashion_rec_loss", fashion_rec_loss)
    cult_trainer.writer.add_scalar("eval/mnist_recc_loss", mnist_rec_loss)
    fashion_env_acc = cult_trainer.env_accuracy(fashion_test_loader, 0)
    mnist_env_acc = cult_trainer.env_accuracy(mnist_test_loader, 1)
    cult_trainer.writer.add_scalar("eval/fashion_env_acc", fashion_env_acc)
    cult_trainer.writer.add_scalar("eval/mnist_env_acc", mnist_env_acc)

epoch: 0, loss=334.8923034667969, rec_loss=323.24334716796875, total_div_loss=11.648744583129883
epoch: 1, loss=294.34661865234375, rec_loss=280.0571594238281, total_div_loss=14.289599418640137
epoch: 2, loss=285.8857727050781, rec_loss=270.50286865234375, total_div_loss=15.382989883422852
epoch: 3, loss=282.7018127441406, rec_loss=266.8443298339844, total_div_loss=15.857505798339844
epoch: 4, loss=281.1229248046875, rec_loss=264.9488220214844, total_div_loss=16.174055099487305
epoch: 5, loss=280.21435546875, rec_loss=263.57806396484375, total_div_loss=16.636117935180664
epoch: 6, loss=279.5182189941406, rec_loss=262.634765625, total_div_loss=16.883302688598633
epoch: 7, loss=278.9308166503906, rec_loss=261.93914794921875, total_div_loss=16.99139976501465
epoch: 8, loss=278.6372985839844, rec_loss=261.5247802734375, total_div_loss=17.112462997436523
epoch: 9, loss=278.3917541503906, rec_loss=261.1748046875, total_div_loss=17.21686553955078
epoch: 0, loss=184.9942626953125, rec_loss=175

  0%|          | 0/7 [00:00<?, ?it/s]

epoch 0 acc tensor(0.7438)
epoch 1 acc tensor(0.7671)
epoch 2 acc tensor(0.7724)
epoch 0 acc tensor(0.7176)
epoch 1 acc tensor(0.7701)
epoch 2 acc tensor(0.7757)


 14%|█▍        | 1/7 [00:23<02:23, 23.99s/it]

epoch 0 acc tensor(0.7400)
epoch 1 acc tensor(0.7533)
epoch 2 acc tensor(0.7582)
epoch 0 acc tensor(0.6762)
epoch 1 acc tensor(0.7087)
epoch 2 acc tensor(0.7198)


...

 71%|███████▏  | 5/7 [02:13<00:54, 27.03s/it]

epoch 0 acc tensor(0.6866)
epoch 1 acc tensor(0.7004)
epoch 2 acc tensor(0.7146)
epoch 0 acc tensor(0.8242)
epoch 1 acc tensor(0.8321)
epoch 2 acc tensor(0.8402)


 86%|████████▌ | 6/7 [02:38<00:26, 26.54s/it]

epoch 0 acc tensor(0.6695)
epoch 1 acc tensor(0.6858)
epoch 2 acc tensor(0.6897)
epoch 0 acc tensor(0.8254)
epoch 1 acc tensor(0.8383)
epoch 2 acc tensor(0.8448)


100%|██████████| 7/7 [03:04<00:00, 26.41s/it]
